# TP 3 : Programmation dynamique


### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Résolution

In [2]:
function solveKnapInstance(filename, showtab)
    price, weight, capacity = readKnaptxtInstance(filename)
    Bestsol=Float64[]

    # Relation de récurrence
    # cas initial C_(i, 0) = 0, ∀ i
    # récurrence C_(i,j) = max{ C_(i−1, j), C_(i−1, j−w_i) + c_i }
    # j : poids maximal permis
    # i : indice du dernier objet considéré
    function C(i, j)
        if j <= 0 || i <= 0
            return 0
        else
            c1 = C(i-1, j)
            if j-weight[i] < 0 # s'il n'y a plus de place, on ne peut pas ajouter l'objet
                return c1
            else # s'il y a de la place, maximisation de la marge
                c2 = C(i-1, j-weight[i]) + price[i]
                return max(c1, c2)
            end
        end
    end

    # Calcul de la meilleure marge
    BestProfit = C(length(weight), capacity)

    # Parcours à rebours à partir de la dernière décision pour obtenir la meilleure solution
    i = length(weight)
    j = capacity
    while i > 1
        # deux choix pour la dernière décision :
        c1 = C(i-1,j) 
        c2 = C(i-1,j-weight[i]) + price[i] 
        if c1 > c2 # on a pas pris l'objet
            j = j
            i = i-1
        else # on a pris l'objet
            push!(Bestsol, i)
            j = j-weight[i]
            i = i-1
        end
    end

    # Affichage des résulats
    if showtab
        print("\t")
        for j in 0:capacity
            print("j=", j, "\t")
        end
        print("\n")
        for i in 1:(length(weight))
            print("i=", i, "\t")
            for j in 0:capacity
                print(C(i,j), "\t")
            end
            print("\n")
        end
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol

end


solveKnapInstance (generic function with 1 method)

In [4]:
INSTANCES = ["InstancesKnapSack/test.opb.txt", "InstancesKnapSack/similar_weights/knapPI_9_50_1000_2_-997.opb.txt", "InstancesKnapSack/profit_ceiling/knapPI_15_20_1000_1_-999.opb.txt", "InstancesKnapSack/weakly_correlated_span/knapPI_12_20_1000_1_-970.opb.txt"]

INSTANCE_N = 3     # entre 1 et 4, pour lancer différents tests
SHOW_TAB = false   # afficher le tableau ?

solveKnapInstance(INSTANCES[INSTANCE_N], SHOW_TAB)


******

Optimal value = 999

Optimal x=[20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0]


(999, [20.0, 19.0, 18.0, 17.0, 16.0, 15.0, 14.0, 13.0, 12.0, 11.0, 10.0, 9.0, 8.0, 7.0, 6.0, 5.0, 4.0, 3.0, 2.0])

### Question 1 : Programmation dynamique
La programmation dynamique consiste à résoudre un problème en résolvant des sous-problèmes de tailles inférieures. Les valeurs obtenues pour les sous-problèmes permettent de résoudre le problème de taille supérieure. La résolution successive de sous-problèmes permet de résoudre le problème initial.

Cette technique nécessite une relation de récurrence entre les valeurs des critères de deux niveaux successifs.

### Question 2 : Cohérence du résultat
Avec la première instance le résultat semblent bien en adéquation avec l'instance résolue : On obtient bien la solution trouvée avec l'algorithme de Branch&Bound implémenté lors du TP2 donc notre solution semblent cohérente.
La solution est x=[4.0, 2.0] pour laquelle la valeur fonction-objectif est 65.

### Question 3 : Fonctionnement de l'algorithme
Nous avons implanté la relation de récurrence $C_{i,j} = max( C_{i−1, j}, C_{i−1, j−w_i} + c_i )$ par la fonction C.

Nous calculons ensuite la valeur optimale de la fonction-objectif en appelant C avec les paramètres du problème. Pour obtenir la solution optimale, nous parcourons l'arbre de décision à rebours.

Pour plus de détails, voir les commentaires du code.

### Question 4 : Test

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| Valeure optimale | 65 | 997 | 999 | 970 | 
| Temps d'exécution (ms) | 4 | 9 | 3 | 5 |

On trouve bien les mêmes solutions que lors du TP2 donc nos résultats sont cohérents

### Question 5 : Comparaison des temps de calcl avec le B&B

Lors du TP 2 nous avions trouvé les temps d'exécution suivants :

**Borne 1**

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| Temps d'exécution (ms) | 21 | 714 | 748 | 82 | 


**Borne 2**

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| Temps d'exécution (ms) | 19 | 257 | 444 | 42 | 

On observe que le temps d'exécution de notre algorithme de programmation dynamique est bien inférieur à la solution apportée par l'algorithme du TP 2.

**Comparaison des rapport de réduction des temps d'exécution par rapport à la borne 1 du Branch & Bound**

| Instance         | 1 | 2 | 3 | 4 | 
|-------------------|---|---|---|---|
| B&B Borne 1 | - | - | - | - | 
| B&B Borne 2 | 10% | 64% | 41% |49% | 
| Progammation Dynamique | 81% | 99% | 99.9% | 99% | 

La programmation dynamique semble ainsi bien plus efficace que la méthode du Branch & Bound pour trouver la solution optimale de ces problèmes.